In [1]:
# needs librosa, anaconda for python 3.
import glob
import os
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.pyplot import specgram
%matplotlib inline
plt.style.use('ggplot')

In [2]:
#extracts mfccs features from audio file
def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name)
    #stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    #chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    #mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    #contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    #tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),
    #sr=sample_rate).T,axis=0)
    return mfccs

In [3]:
#read csv file into pandas dataframe
data = pd.read_csv('UrbanSound8K/metadata/UrbanSound8K.csv')
data

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing


In [40]:
data_dir='./UrbanSound8K/audio'

def mfccs_parser(row):
    # function to load files and extract features
    features, labels = np.empty((0,40)), np.empty(0)
    fold='fold'+str(row.fold)
    file=row.slice_file_name
    file_name = os.path.join(data_dir, fold, file)
    file_name=file_name.replace("\\", "/")
    #print(file_name)
   # handle exception to check if there isn't a file which is corrupted
    try:
        X, sample_rate = librosa.load(file_name)
        # we extract mfcc feature from data
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0) 
      
    except Exception as e:
        print("Error encountered while parsing file: ")
        return None
    #if I return just mfccs expected shape error in DF.
    return [mfccs]
    

In [41]:
#temp = data.apply(mfccs_parser, axis=1)
#temp
data['mfccs']=data.apply(mfccs_parser, axis=1)
data
### mfccs has became list of lists, how to avoid that ????

,slice_file_name,fsID,start,end,salience,fold,classID,class,mfccs
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark,"[[-209.986106799, 63.4275064272, -123.69955651..."
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing,"[[-417.341815342, 100.15644544, -43.8545510775..."
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing,"[[-452.779325426, 113.231555902, -38.488149466..."
